In [1]:
%reload_ext cython

In [ ]:
! pip install stop_words

In [2]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


In [21]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [3]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer



#Function to make document, word matricies for LDA#
def make_word_matrix(corpus, needToSplit):
    
    
    #Check to make sure NeedToSplit argument is 0 or 1
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    #Define stop words
    stopWords = get_stop_words('english')
    
    
    #Initialize stemmer
    p_stemmer = PorterStemmer()

    
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    
    #Check to make sure that dictionary isn't empty#
    if M ==0:
        print("Input dictionary is empty")
        return;
    
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        if isinstance(corpus[i], str) != True:
            print("Corpus input is not a string")
            return;
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        # stem tokens
        text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
            #Remove stop words#
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]
            
            #remove stop words
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [4]:
corpusMatrix = make_word_matrix(docs, 1)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference**

In [5]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  The output of this function are the matricies gamma and phi, where gamma (k vector) is the Dirichlet paramteters and the matrix phi (N x k, where k is the number of topics) are the multinomial paramters.  See page 1004 of paper for derivation.

In [6]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

In [26]:
k = 3
np.random.seed(7)
alphaOld = 10*np.random.rand(k)
V = corpusMatrix[0][0].shape[1]
betaOld = np.random.rand(k, V)
betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]

In [27]:
%%time 
for i in range(0,100):
    phi, gamma = Estep(k, 0, alphaOld, betaOld, corpusMatrix, 0.1)

Wall time: 112 ms


**Parameter Estimation**

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 for derivation.  

The alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the Mstep() function maximizes for alpha and beta.

In [7]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return alphaNew

In [8]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)

Check that gsl and scipy results are similar
----

Cython code for updateing $\alpha$ and $\beta$
----

In [9]:
%%cython -a -lgsl

import cython
import numpy as np
cimport numpy as np
import scipy
from cython_gsl cimport *

#Update alpha using linear Newton-Rhapson Method#

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:] cy_alphaUpdate(int k, int M, double[:] alphaOld, double[:, :] gamma, double tol):
    cdef double[:] h = np.zeros(k)
    cdef double[:] g = np.zeros(k)
    cdef double[:] alphaNew = np.zeros(k)

    cdef int converge
    cdef int i, d
    cdef double docSum
    cdef double[:] step = np.zeros(k)
    cdef double c, s1, s2

    cdef double alpha_sum = 0
    for i in range(k):
        alpha_sum += alphaOld[i]

    converge = 0
    while converge == 0:
        z = -gsl_sf_psi_n(1, alpha_sum)

        s1 = 0
        s2 = 1.0/z
        for i in range(0, k):
            docSum = 0
            for d in range(M):
                docSum += gsl_sf_psi(gamma[d,i]) - gsl_sf_psi(sum(gamma[d]))
            g[i] = M*(gsl_sf_psi(alpha_sum) - gsl_sf_psi(alphaOld[i])) + docSum
            h[i] = M*gsl_sf_psi_n(1, alphaOld[i])
            
            s1 += g[i]/h[i]
            s2 += 1.0/h[i]
        c = s1/s2

        for i in range(k):
            step[i] = (g[i] - c)/h[i]
        # step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            for i in range(k):
                alphaNew[i] = alphaOld[i] + step[i]
            alphaOld = alphaNew

    return alphaNew

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:, :] cy_betaUpdate(int k, int M, long[:] phi_1, double[:, :, :] phi_2, double[:, :] gamma, 
                              long[:] c_1, double[:, :, :] c_2, double tol):

    cdef int i, j, d, n
    cdef int Nd
    cdef double wordSum

    #Calculate beta#
    cdef int V = c_2[0].shape[1]
    cdef double[:, :] beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(M):
                Nd = c_1[d] # c[d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi_2[d, n,i]*c_2[d, n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]

    return beta

Check Cython functions against original functions
----

### Prepare funciton arguments

In [11]:
k = 3
M = 4
phi = [np.array([[ 0.38340797,  0.30327435,  0.31331768],
       [ 0.5703127 ,  0.17742331,  0.25226399],
       [ 0.3176045 ,  0.43558053,  0.24681497]]), np.array([[ 0.55072546,  0.18583053,  0.26344401],
       [ 0.30048528,  0.44698122,  0.2525335 ],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608]]), np.array([[ 0.0623253 ,  0.24856995,  0.68910475],
       [ 0.63063084,  0.08148213,  0.28788703],
       [ 0.12858014,  0.39114008,  0.48027977]]), np.array([[ 0.66366397,  0.08257639,  0.25375964],
       [ 0.77627072,  0.10684157,  0.11688771],
       [ 0.75940435,  0.23665379,  0.00394186],
       [ 0.02553949,  0.62915851,  0.345302  ],
       [ 0.14168349,  0.41504784,  0.44326867]])]
gamma = np.array([np.array([ 9.71859516,  6.63758257,  7.30234283]), np.array([ 10.58244789,   7.50992466,   8.56614802]), np.array([ 9.26880627,  6.44249655,  7.94721775]), np.array([ 10.813832  ,   7.19158249,   7.65310607])])
alphaOld = np.array([ 8.44726999 , 5.72130438,  6.48994619])
c = np.array([np.array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])])
tol = 0.1

In [9]:
phi

[array([[ 0.38340797,  0.30327435,  0.31331768],
        [ 0.5703127 ,  0.17742331,  0.25226399],
        [ 0.3176045 ,  0.43558053,  0.24681497]]),
 array([[ 0.55072546,  0.18583053,  0.26344401],
        [ 0.30048528,  0.44698122,  0.2525335 ],
        [ 0.32099179,  0.28895213,  0.39005608],
        [ 0.32099179,  0.28895213,  0.39005608],
        [ 0.32099179,  0.28895213,  0.39005608],
        [ 0.32099179,  0.28895213,  0.39005608]]),
 array([[ 0.0623253 ,  0.24856995,  0.68910475],
        [ 0.63063084,  0.08148213,  0.28788703],
        [ 0.12858014,  0.39114008,  0.48027977]]),
 array([[ 0.66366397,  0.08257639,  0.25375964],
        [ 0.77627072,  0.10684157,  0.11688771],
        [ 0.75940435,  0.23665379,  0.00394186],
        [ 0.02553949,  0.62915851,  0.345302  ],
        [ 0.14168349,  0.41504784,  0.44326867]])]

In [10]:
r = len(phi)
phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
n = max(phi_1)
p = phi[0].shape[1]
phi_2 = np.zeros((r, n, p))
for i, j in enumerate(phi_1):
    phi_2[i, :j, :] = phi[i]

In [11]:
c

array([ array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
       array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
       array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
       array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.

In [12]:
r = len(c)
c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
n = max(c_1)
p = c[0].shape[1]
c_2 = np.zeros((r, n, p))
for i, j in enumerate(c_1):
    c_2[i, :j, :] = c[i]

### Check

In [ ]:
a, b = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)

In [ ]:
a

In [ ]:
alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
alphaNew

In [ ]:
b

In [ ]:
betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))
betaNew

Is there any speed-up?
----

In [ ]:
%timeit a, b = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)

In [ ]:
%%timeit
alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))

**LDA Function:**
Finally, we implement the entire Latent Dirichlet Allocation method in the LDA function, which takes as its arguments k (the number of topics), D (the number of documents in the corpus), a corpus matrix (the output from make_word_matrix above) and a tolerance (which sets the convergence criteria for the while loops).  For each document d, the function runs until the alpha or beta parameters converge, by first running the E step and then the M step for each document separately.  The final values of phi, gamma, alpha and beta are returned for all D documents in a list.

In [10]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, corpusMatrix, tol):

    # create rectangular matrices for c
    c = corpusMatrix[0]
    r_c = len(c)
    c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
    n_c = max(c_1)
    p_c = c[0].shape[1]
    c_2 = np.zeros((r_c, n_c, p_c))
    for i, j in enumerate(c_1):
        c_2[i, :j, :] = c[i]
    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        # create rectangular matrices for phi
        r = len(phi)
        phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
        n = max(phi_1)
        p = phi[0].shape[1]
        phi_2 = np.zeros((r, n, p))
        for i, j in enumerate(phi_1):
            phi_2[i, :j, :] = phi[i]

                
        gamma = np.array(gamma)

        alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
        betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [11]:
%%time

LDA(3, corpusMatrix, 0.1)

CPU times: user 4.79 ms, sys: 1.92 ms, total: 6.71 ms
Wall time: 18.3 ms


[[[array([[ 0.16017836,  0.133443  ,  0.70637864],
          [ 0.14653824,  0.06965857,  0.78380319],
          [ 0.48664435,  0.11127472,  0.40208092]]),
   array([[ 0.14683332,  0.07555678,  0.77760991],
          [ 0.48412656,  0.11983094,  0.39604251],
          [ 0.181337  ,  0.17619375,  0.64246925],
          [ 0.181337  ,  0.17619375,  0.64246925],
          [ 0.181337  ,  0.17619375,  0.64246925],
          [ 0.181337  ,  0.17619375,  0.64246925]]),
   array([[ 0.01377702,  0.0842106 ,  0.90201238],
          [ 0.17763066,  0.23309834,  0.589271  ],
          [ 0.36769147,  0.40128441,  0.23102412]]),
   array([[ 0.19540238,  0.25368431,  0.55091331],
          [ 0.15942572,  0.26094724,  0.57962704],
          [ 0.60741577,  0.25976583,  0.1328184 ],
          [ 0.06276253,  0.19547009,  0.74176738],
          [ 0.38259844,  0.41309927,  0.20430229]])],
  array([[  4.23175185,   3.10827825,  10.01747679],
         [  4.79469876,   3.69406467,  11.86874346],
         [  3.9974

In [12]:
#function to return the most probable words
#p is the number of words you want returned for each topic
def mostCommon(beta, wordList, p):
    k = beta.shape[0]
    topicWords = []
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    for i in range(0, k):
        document = betaDF.loc[i,:]
        document.sort(1, ascending = 0)
        mostCommon = pd.DataFrame(document[0:p])
        topicWords.append(mostCommon)
    return(topicWords)

**State of the Union Example - Only Used for Timing of Functions**

In order to determine the initial performance time of our algorithm, we implemented LDA on a small corpus of eight documents. These documents are each of the State of the Union Adresses delivered by President Clinton during his time in office. These were found in the nltk Python package.

In [ ]:
import nltk
#nltk.download("state_union")
from nltk.corpus import state_union

In [ ]:
#creating a diciontary of the state of the Union Addresses for Clinton
fileNames = state_union.fileids()
Clinton = fileNames[50:58]

ClintonSOTU = {}
for i in range(0, len(Clinton)):
    ClintonSOTU[Clinton[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(Clinton[i]))

In [ ]:
# fileNames = state_union.fileids()
# SOTU = fileNames

# PresSOTU = {}
# for i in range(0, len(SOTU)):
#     PresSOTU[SOTU[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(SOTU[i]))

In [ ]:
# np.random.seed(17)
# PresCorp = make_word_matrix(PresSOTU, 0)
# ResultsAll = LDA(5, PresCorp, 0.1)

In [ ]:
%%time
np.random.seed(7)
ClintCorp = make_word_matrix(ClintonSOTU, 0)
Results = LDA(3, ClintCorp, 0.1)

In [ ]:
mostCommon(ResultsAll[0][3], PresCorp[1], 2)

In [ ]:
# from nltk.corpus import gutenberg
# gutenberg.fileids()

In [ ]:
# bookNames = gutenberg.fileids()
# indicies = [0,7,15]
# books = [bookNames[i] for i in indicies]
# books

In [ ]:


# booksInput = {}
# for i in range(0, len(books)):
#     booksInput[books[i].rsplit('.', 1)[0]] = list(nltk.corpus.gutenberg.words(books[i]))

In [ ]:
# np.random.seed(77)
# bookCorp = make_word_matrix(booksInput, 0)
# print("LDA")
# ResultsBooks = LDA(5, bookCorp, 0.1)

## Sources - LDA Function Test:
http://www.gutenberg.org/cache/epub/4/pg4.txt
http://www.gutenberg.org/files/16780/16780-h/16780-h.html
http://www.americanrhetoric.com/speeches/mlkihaveadream.htm
http://www.americanrhetoric.com/speeches/jfkinaugural.htm

In [20]:
file = open('Gettysburg.txt', 'r')
Gettysburg = file.read()
file2 = open("Dec_of_Independence.txt", 'r')
Declaration = file2.read()
file3 = open("MLK_Dream.txt", 'r')
MLK = file3.read()
file4 = open("JFK_Inauguration.txt", 'r')
JFK = file4.read()


In [21]:
speechDocs = collections.OrderedDict()
speechDocs["s1"] = Gettysburg
speechDocs["s2"] = Declaration
speechDocs["s3"] = MLK
speechDocs["s4"] = JFK

In [22]:
speechCorp = make_word_matrix(speechDocs, 1)

In [23]:
np.random.seed(17)
#ResultsSpeech = LDA(3, speechCorp, 0.1)
ResultsSpeech2 = LDA(10, speechCorp, 0.1)

In [24]:
mostCommon(ResultsSpeech2[0][3], speechCorp[1], 5)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[             0
 will  0.020268
 can   0.018629
 one   0.010692
 live  0.010239
 men   0.009886,                 1
 one      0.017133
 us       0.015581
 freedom  0.015379
 govern   0.010825
 power    0.009820,               2
 us     0.014927
 everi  0.014412
 new    0.012566
 let    0.012358
 state  0.011810,                 3
 will     0.029280
 world    0.017139
 new      0.013814
 freedom  0.010950
 today    0.010817,                4
 right   0.018156
 let     0.016769
 will    0.015490
 peopl   0.013783
 nation  0.013478,                5
 right   0.020108
 let     0.019590
 us      0.018422
 nation  0.016359
 one     0.013549,                6
 will    0.023728
 let     0.019589
 free    0.016950
 nation  0.014192
 new     0.013871,                7
 will    0.016827
 can     0.015872
 nation  0.015305
 right   0.012156
 us      0.011036,                 8
 let      0.023946
 will     0.022135
 freedom  0.016181
 nation   0.014928
 everi    0.013984,                 9
 let     

## Color Code Text by Topic
<span style="color:red">Topic 1</span> 

<span style="color:blue">Topic 2</span> 

<span style="color:green">Topic 3</span> 

If a specific word is in more than one topic, I assigned it to that topic that gave that word the highest probability.

When in the <span style="color:blue">Course</span>  of <span style="color:green">human</span> <span style="color:red">events</span> , it <span style="color:red">becomes</span> <span style="color:green">necessary</span> for <span style="color:green">one</span> <span style="color:green">people</span> to <span style="color:green">dissolve</span> the <span style="color:green">political</span> <span style="color:red">bands</span> 
which have <span style="color:blue">connected</span> them with <span style="color:green">another</span>, and to <span style="color:blue">assume</span>, <span style="color:blue">among</span> the <span style="color:green">Powers</span> of the <span style="color:green">earth</span> , the <span style="color:green">separate</span> and <span style="color:green">equal</span> 
<span style="color:blue">station</span> to which the <span style="color:blue">Laws</span> of <span style="color:blue">Nature</span> and of <span style="color:blue">Nature's</span> <span style="color:green">God</span> <span style="color:green">entitle</span> them, a <span style="color:red">decent</span> <span style="color:red">respect</span> to the <span style="color:red">opinions</span> of 
<span style="color:red">mankind</span> <span style="color:green">requires</span> that they should 
<span style="color:green">declare</span> the <span style="color:red">causes</span> which <span style="color:red">impel</span> them to the <span style="color:green">separation</span>.

We <span style="color:blue">hold</span> these <span style="color:green">truths</span> to be <span style="color:blue">self-evident</span>, that all <span style="color:blue">men</span> are <span style="color:red">created</span> <span style="color:green">equal</span>, that they are <span style="color:blue">endowed</span> by their 
<span style="color:green">Creator</span> with <span style="color:blue">certain unalienable Rights</span>, that <span style="color:blue">among</span> these are <span style="color:blue">Life</span>, <span style="color:green">Liberty</span>, and the <span style="color:red">pursuit</span> of <span style="color:blue">Happiness</span>.

In [36]:
decIntro = """When in the Course of human events, it becomes necessary for one people to dissolve the political bands 
which have connected them with another, and to assume, among the Powers of the earth, the separate and equal 
station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of 
mankind requires that they should 
declare the causes which impel them to the separation.

We hold these truths to be self-evident, that all men are created equal, that they are endowed by their 
Creator with certain unalienable Rights, that among these are Life, Liberty, and the pursuit of Happiness."""
#colorCode(ResultsSpeech[0][3], speechCorp[1], decIntro).T

In [40]:
GSample = """It is for us the living, rather, to be dedicated here to the unfinished
work which they who fought here have thus far so nobly advanced.
It is rather for us to be here dedicated to the great task remaining
before us. . .that from these honored dead we take increased devotion
to that cause for which they gave the last full measure of devotion. . .
that we here highly resolve that these dead shall not have died in vain. . .
that this nation, under God, shall have a new birth of freedom. . .
and that government of the people. . .by the people. . .for the people. . .
shall not perish from this earth."""

In [35]:
def colorCode(beta, wordList, inputWords):
    
    #Split text, convert to lowercase and remove punctuation and stop words
    stopWords = get_stop_words('english')
    table = dict.fromkeys(map(ord, string.punctuation))
    text = inputWords.translate(table)
    words = text.lower().split()
    words = [word for word in words if word not in stopWords]
    p_stemmer = PorterStemmer()
    words = [p_stemmer.stem(word) for word in words]
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    
    
    k = len(words)
    topicWords = np.zeros((1,k))
    topicWords = pd.DataFrame(topicWords,  columns=words)

    for i in range(0, k):
        document = np.array(betaDF.loc[:,words[i]])
        topicWords.iloc[:,i] = np.argmax(document)
    return(topicWords)

In [ ]:
mostCommon(ResultsSpeech[0][3], speechCorp[1], 25)

In [1]:
#colorCode(ResultsSpeech2[0][3], speechCorp[1], GSample).T

<span style="color:red">Topic 0</span> 
<span style="color:blue">Topic 1</span> 
<span style="color:green">Topic 2</span> 
<span style="color:darkmagenta">Topic 3</span> 
<span style="color:orange">Topic 4</span> 
<span style="color:lightseagreen">Topic 5</span> 
<span style="color:darkblue">Topic 6</span> 
<span style="color:magenta">Topic 7</span> 
<span style="color:cyan">Topic 8</span> 
<span style="color:chartreuse">Topic 9</span> 

## Declaration

When in the <span style="color:chartreuse">Course</span> of <span style="color:lightseagreen">human</span> <span style="color:red">events</span>, it <span style="color:red">becomes</span> <span style="color:green">necessary</span> for <span style="color:blue">one</span> <span style="color:orange">people</span> to <span style="color:darkmagenta">dissolve</span> the <span style="color:magenta">political</span> <span style="color:cyan">bands</span> 
which have <span style="color:darkblue">connected</span> them with <span style="color:darkmagenta">another</span>, and to <span style="color:darkmagenta">assume</span>, <span style="color:blue">among</span> the <span style="color:chartreuse">Powers</span> of the <span style="color:chartreuse">earth</span>, the <span style="color:chartreuse">separate</span> and <span style="color:orange">equal</span> 
<span style="color:lightseagreen">station</span> to which the <span style="color:orange">Laws</span> of <span style="color:lightseagreen">Nature</span> and of <span style="color:lightseagreen">Nature's</span> <span style="color:darkmagenta">God</span> <span style="color:magenta">entitle</span> them, a <span style="color:lightseagreen">decent</span> <span style="color:green">respect</span> to the <span style="color:darkmagenta">opinions</span> of 
<span style="color:lightseagreen">mankind</span> <span style="color:magenta">requires</span> that they should 
<span style="color:lightseagreen">declare</span> the <span style="color:orange">causes</span> which <span style="color:lightseagreen">impel</span> them to the <span style="color:chartreuse">separation</span>.

We <span style="color:magenta">hold</span> these <span style="color:blue">truths</span> to be <span style="color:chartreuse">self-evident</span>, that all <span style="color:red">men</span> are <span style="color:chartreuse">created</span> <span style="color:orange">equal</span>, that they are <span style="color:blue">endowed</span> by their 
<span style="color:lightseagreen">Creator</span> with <span style="color:orange">certain</span> <span style="color:chartreuse">unalienable</span> <span style="color:lightseagreen">Rights</span>, that <span style="color:blue">among</span> these are <span style="color:red">Life</span>, <span style="color:darkblue">Liberty</span>, and the <span style="color:cyan">pursuit</span> of <span style="color:magenta">Happiness</span>.

## Gettysburg

It is for <span style="color:lightseagreen">us</span> the <span style="color:red">living</span>, <span style="color:orange">rather</span>, to be <span style="color:orange">dedicated</span> here to the <span style="color:orange">unfinished</span>
<span style="color:magenta">work</span> which they who <span style="color:cyan">fought</span> here have <span style="color:orange">thus</span> <span style="color:darkblue">far</span> so <span style="color:lightseagreen">nobly</span> <span style="color:lightseagreen">advanced</span>.
It is <span style="color:orange">rather</span> for <span style="color:lightseagreen">us</span> to be here <span style="color:orange">dedicated</span> to the <span style="color:red">great</span> <span style="color:lightseagreen">task</span> <span style="color:chartreuse">remaining</span>
before <span style="color:lightseagreen">us</span>. . .that from these <span style="color:red">honored</span> <span style="color:darkblue">dead</span> we <span style="color:magenta">take</span> <span style="color:orange">increased</span> <span style="color:green">devotion</span>
to that <span style="color:orange">cause</span> for which they <span style="color:orange">gave</span> the <span style="color:orange">last</span> <span style="color:blue">full</span> <span style="color:magenta">measure</span> of <span style="color:green">devotion</span>. . .
that we here <span style="color:darkblue">highly</span> <span style="color:darkblue">resolve</span> that these <span style="color:darkblue">dead</span> <span style="color:magenta">shall</span> not have <span style="color:darkblue">died</span> in <span style="color:orange">vain</span>. . .
that this <span style="color:lightseagreen">nation</span>, under <span style="color:darkmagenta">God</span>, <span style="color:magenta">shall</span> have a <span style="color:darkblue">new</span> <span style="color:lightseagreen">birth</span> of <span style="color:cyan">freedom</span>. . .
and that <span style="color:darkblue">government</span> of the <span style="color:orange">people</span>. . .by the <span style="color:orange">people</span>. . .for the <span style="color:orange">people</span>. . .
<span style="color:magenta">shall</span> not <span style="color:lightseagreen">perish</span> from this <span style="color:chartreuse">earth</span>.

In [ ]:
#pip install --pre line-profiler
#pip install psutil
# pip install memory_profiler

In [ ]:
#creating profile files for the construction of the word matrix and LDA for SOTU example
%prun -q -D ClintonWordMatrix.prof make_word_matrix(ClintonSOTU,0)
%prun -q -D ClintonLDA.prof LDA(3, ClintCorp, 0.1, 0)

In [ ]:
#printing profile results
import pstats
wordMatrixProfile = pstats.Stats('ClintonWordMatrix.prof')
LDAProfile = pstats.Stats('ClintonLDA.prof')
wordMatrixProfile.print_stats()
LDAProfile.print_stats()
pass

**Movie Clustering** 

In addition to text analysis, LDA can be implemented for other problems that have the same structure as a text corpora. In this section, we use LDA in order to predict movies preferred by users of the site MovieLens.com. The goal is to predict another movie that the user likes based on the movies that they have already said that they prefer. The definition of preferred here is that the user rated the movie four or five, where five is the maxiumum possible rating. Users who gave a preferred rating to at least 50 movies were used. A training set of 2,535 users was used to fit the model. The model is then shown all but one of the preferred movies for the users in the test data, and this is used to determine the last movie that the user preferred. Thus, it is possible to evaluate the performance of the model.

In [267]:
#reading in the data
import pandas as pd
Ratings = pd.read_csv("RatingsDataClean.csv")
movieData = pd.read_csv("MovieReferenceTable.csv", encoding='cp1252')


In [2]:
#Ratings.head()

In [191]:
#making a dictionary for the training and testing data
Users = list(set(Ratings['userID']))
RatingsDict = {}
for i in range(0, len(Users)):
    movies = list(map(str, list(pd.DataFrame(Ratings[Ratings['userID'] == Users[i]])['movieID'])))
    RatingsDict[i] = ' '.join(movies)

In [3]:
# for k, v in RatingsDict.items():
#     print(k,v)

In [194]:
#running LDA analysis to get parameters for movie training data
import random

random.seed(77)
movieMatrix = make_word_matrix(RatingsDict, 1)
movieOutput = LDA(5, movieMatrix, 0.1)

In [196]:
movieTopics = mostCommon(movieOutput[0][3], movieMatrix[1], 15)
movieList = []
for i in range(0,5):
    movieList.append(list(np.array(movieTopics[i].index, dtype = int)))
movieList
#list(movieData.ix[:,"Name"].head())
#movieData.head()
#movieData.shape
#movieData.head()

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[[260,
  1197,
  1265,
  318,
  1210,
  1036,
  480,
  1270,
  588,
  589,
  2028,
  2396,
  1200,
  2716,
  3897],
 [1240,
  1196,
  356,
  1198,
  2997,
  2762,
  527,
  593,
  318,
  1197,
  2716,
  2858,
  260,
  1097,
  1210],
 [1198,
  858,
  2571,
  3751,
  1193,
  593,
  2028,
  110,
  480,
  2791,
  2396,
  2959,
  1225,
  2355,
  1259],
 [1196,
  2858,
  318,
  2571,
  589,
  1097,
  527,
  457,
  593,
  2804,
  1214,
  1094,
  594,
  2716,
  1356],
 [2858,
  3578,
  2571,
  1196,
  2762,
  2396,
  260,
  1270,
  110,
  1358,
  480,
  1221,
  908,
  2947,
  2028]]

In [197]:
#how to get the movie names that match the IDs
#replace testList with the list for each cluster
#list(movieData[movieData['movieID'].isin(movieList)]["Name"])
movieCluster = []
for i in range(0,5):
    movieCluster.append(movieData[movieData['movieID'].isin(movieList[i])]["Name"])
movieCluster

#movieData[movieData['movieID'].isin(movieList[0])]["Name"]

[9       Star Wars:  Episode IV - A New Hope (1977): Ac...
 12      Star Wars:  Episode VI - Return of the Jedi (1...
 41      Terminator 2:  Judgment Day (1991): Action|Sci...
 495                      Shawshank Redemption, The (1994)
 648                                  Jurassic Park (1993)
 755                                        Aladdin (1992)
 1169                                      Die Hard (1988)
 1318                           Princess Bride, The (1987)
 1321                                        Aliens (1986)
 1381                                 Groundhog Day (1993)
 1386                            Back to the Future (1985)
 2042                           Saving Private Ryan (1998)
 2388                           Shakespeare in Love (1998)
 2692                                  Ghostbusters (1984)
 3831                                 Almost Famous (2000)
 Name: Name, dtype: object,
 7       Star Wars:  Episode V - The Empire Strikes Bac...
 9       Star Wars:  Episode

In [254]:

betaDF = movieOutput[0][3]

k = len(movieMatrix[1])
topicWords = np.zeros((1,k))
topicWords = pd.DataFrame(topicWords,  columns=movieMatrix[1])

for i in range(0, k):
    document = np.array(betaDF[:,i])
    topicWords.iloc[:,i] = np.argmax(document)

topicWords = topicWords.T

topicWords['movieID'] = topicWords.index.astype(int)

topicWords

,0,movieID
1,4,1
10,0,10
100,1,100
1002,2,1002
1003,3,1003
1005,0,1005
1007,3,1007
1008,3,1008
1009,0,1009
101,2,101


In [268]:
# test = movieData
# test.head()
# #pd.merge(topicWords, test, how='left', on = "movieID")
movieData = topicWords.merge(movieData, on = "movieID", how = "left")
movieData.head()

,0,movieID,Unnamed: 0,Name,Genre
0,4,1,0,Toy Story (1995),Animation|Children's|Comedy
1,0,10,9,GoldenEye (1995),Action|Adventure|Thriller
2,1,100,98,City Hall (1996),Drama|Thriller
3,2,1002,989,Ed's Next Move (1996),Comedy
4,3,1003,990,Extreme Measures (1996),Drama|Thriller


In [270]:
movieData.columns = ['cluster', 'movieID', 'UserID', "Name", "Genre"]
movieData.head()

,cluster,movieID,UserID,Name,Genre
0,4,1,0,Toy Story (1995),Animation|Children's|Comedy
1,0,10,9,GoldenEye (1995),Action|Adventure|Thriller
2,1,100,98,City Hall (1996),Drama|Thriller
3,2,1002,989,Ed's Next Move (1996),Comedy
4,3,1003,990,Extreme Measures (1996),Drama|Thriller


In [274]:
movieData[movieData['cluster'] == 0].head(10)

,cluster,movieID,UserID,Name,Genre
1,0,10,9,GoldenEye (1995),Action|Adventure|Thriller
5,0,1005,992,D3: The Mighty Ducks (1996): Children's|Comed...,Children's|Comedy
8,0,1009,996,Escape to Witch Mountain (1975),Adventure|Children's|Fantasy
10,0,1010,997,"Love Bug, The (1969)",Children's|Comedy
12,0,1012,999,Old Yeller (1957),Children's|Drama
17,0,1017,1004,Swiss Family Robinson (1960),Adventure|Children's
21,0,1021,1008,Angels in the Outfield (1994),Children's|Comedy
24,0,1024,1011,"Three Caballeros, The (1945)",Animation|Children's|Musical
32,0,1033,1020,"Fox and the Hound, The (1981)",Animation|Children's
33,0,1034,1021,Freeway (1996),Crime


In [275]:
movieData[movieData['cluster'] == 1].head(10)

,cluster,movieID,UserID,Name,Genre
2,1,100,98,City Hall (1996),Drama|Thriller
15,1,1015,1002,Homeward Bound: The Incredible Journey (1993)...,Adventure|Children's
18,1,1018,1005,That Darn Cat! (1965),Children's|Comedy|Mystery
31,1,1032,1019,Alice in Wonderland (1951),Animation|Children's|Musical
36,1,1037,1024,"Lawnmower Man, The (1992)",Action|Sci-Fi|Thriller
39,1,1041,1028,Secrets & Lies (1996),Drama
41,1,1043,1030,To Gillian on Her 37th Birthday (1996),Drama|Romance
44,1,1047,1034,"Long Kiss Goodnight, The (1996)",Action|Thriller
49,1,1053,1039,Normal Life (1996),Crime|Drama
57,1,1063,1049,Johns (1996),Drama


In [276]:
movieData[movieData['cluster'] == 2].head(10)

,cluster,movieID,UserID,Name,Genre
3,2,1002,989,Ed's Next Move (1996),Comedy
9,2,101,99,Bottle Rocket (1996),Comedy
11,2,1011,998,Herbie Rides Again (1974),Adventure|Children's|Comedy
13,2,1013,1000,"Parent Trap, The (1961)",Children's|Drama
16,2,1016,1003,"Shaggy Dog, The (1959)",Children's|Comedy
25,2,1025,1012,"Sword in the Stone, The (1963)",Animation|Children's
28,2,1029,1016,Dumbo (1941),Animation|Children's|Musical
30,2,1031,1018,Bedknobs and Broomsticks (1971),Adventure|Children's|Musical
34,2,1035,1022,"Sound of Music, The (1965)",Musical
40,2,1042,1029,That Thing You Do! (1996),Comedy


In [277]:
movieData[movieData['cluster'] == 3].head(10)

,cluster,movieID,UserID,Name,Genre
4,3,1003,990,Extreme Measures (1996),Drama|Thriller
6,3,1007,994,"Apple Dumpling Gang, The (1975)",Children's|Comedy|Western
7,3,1008,995,"Davy Crockett, King of the Wild Frontier (1955)",Western
20,3,1020,1007,Cool Runnings (1993),Comedy
38,3,104,102,Happy Gilmore (1996),Comedy
46,3,105,103,"Bridges of Madison County, The (1995)",Drama|Romance
51,3,1057,1043,Everyone Says I Love You (1996),Comedy|Musical|Romance
54,3,106,104,Nobody Loves Me (Keiner liebt mich) (1994),Comedy|Drama
61,3,1068,1054,Crossfire (1947),Crime|Film-Noir
68,3,1079,1063,"Fish Called Wanda, A (1988)",Comedy


In [278]:
movieData[movieData['cluster'] == 4].head(10)

,cluster,movieID,UserID,Name,Genre
0,4,1,0,Toy Story (1995),Animation|Children's|Comedy
14,4,1014,1001,Pollyanna (1960),Children's|Comedy|Drama
19,4,1019,1006,"20,000 Leagues Under the Sea (1954)",Adventure|Children's|Fantasy|Sci-Fi
22,4,1022,1009,Cinderella (1950),Animation|Children's|Musical
23,4,1023,1010,Winnie the Pooh and the Blustery Day (1968),Animation|Children's
26,4,1027,1014,Robin Hood: Prince of Thieves (1991): Drama: ...,Drama
27,4,1028,1015,Mary Poppins (1964),Children's|Comedy|Musical
29,4,1030,1017,Pete's Dragon (1977),Adventure|Animation|Children's|Musical
56,4,1061,1047,Sleepers (1996),Crime|Drama
59,4,1066,1052,Shall We Dance? (1937),Comedy|Musical|Romance


## Future Steps

- Compare output to Python package
- Test on corpus in paper
- Model topics on different corpus
- Time and optimize (use Cython, quite slow now)
- Run collaborative filtering
- Compare to Gibbs sampling

## Compare our output to Python Package

https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

In [ ]:
#! pip install gensim

In [ ]:
p_stemmer = PorterStemmer()
for i in docs:
    stemmed_tokens = p_stemmer.stem(i)

In [ ]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "The quick brown fox"
doc_b = "Brown fox jumps over the jumps jumps jumps"
doc_c =  "The the the lazy dog elephant."
doc_d = "The the the the the dog peacock lion tiger elephant"


# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, iterations = 1)

In [ ]:
print(ldamodel.print_topics(num_topics=3, num_words=4))

In [ ]:
beta =  LDATest(3, corpusMatrix, 0.1, 1)[0][3]

In [ ]:
mostCommon(beta, corpusMatrix[1], 4)

In [ ]:
corpusMatrix[1]

In [ ]:
beta

In [ ]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDATest(k, corpusMatrix, tol, needToSplit):

    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    iterations = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    

    phi = []
    gamma = []
    #looping through the number of documents
    for d in range(0,M): #M is the number of documents
        phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
        phi.append(phiT)
        gamma.append(gammaT)

    alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)


    converge =0
    alphaOld = alphaNew
    betaOld = betaNew
    iterations += iterations
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output